### Criação/generalização dos consumos normalizados para o ano todo

Obtivemos, para o estudo de caso para a região de Coimbra (pré-generalização para o país via APIs) os dados de consumos para os Códigos Postais de 4 e de 7 dígitos em ficheiros CSVs a partir da plataforma da E-Redes.

Por conta das limitações apresentadas pelos dados disponibilizados pela E-Redes (com os dados horários para os Códigos Postais de 7 dígitos disponibilizados apenas para o mês de Fevereiro), tivemos de generalizar e extrapolar os valores para o resto do ano. Para isso, criamos funções que realizassem o processo de forma simplificada.

Nossa decisão foi por generalizar os consumos, para os Códigos Postais de 7 dígitos, a partir dos valores observados para os Códigos Postais de 4 dígitos (com dados disponíveis para 11 meses do ano). Para evitar que o real perfil do consumo ao longo do dia no Código Postal de 7 dígitos fosse "escondido" pelos padrões horários para o agregado do Código de 4 Dígitos correspondente, consolidamos os valores diários para os Códigos Postais de 4 dígitos; a seguir, normalizamos esses valores em relação a uma data baseline (10 de fevereiro, segunda-feira); para obter os valores para os demais meses, multiplicamos os valores horários observados nessa data pelo fator multiplicador correspondente ao dia.

A criação desses fatores normalizados foi input fundamental para o processo final de generalização para todo o país, uma vez que era - em paralelo ao pré-processamento dos PTDs, feito via QGIS - o pré-requisito fundamental para possibilitar a automatização caso-a-caso para qualquer ponto do país.

In [1]:
import pandas as pd
import numpy as np

In [2]:
def preparar_dados_consumo(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # 1) Conversão forte para datetime
    # Garante que ambas estão em formato string sem espaços
    df['Data'] = df['Data'].astype(str).str.strip()
    df['Hora'] = df['Hora'].astype(str).str.strip()

    # Monta a string completa e converte para datetime
    df['Data/Hora'] = pd.to_datetime(df['Data'] + ' ' + df['Hora'], errors='coerce')

    # 2) Limpa linhas inválidas e cria auxiliares
    df = df.dropna(subset=['Data/Hora'])
    df['Mês']  = df['Data/Hora'].dt.month
    df['Dia']  = df['Data/Hora'].dt.day
    df['Hora'] = df['Data/Hora'].dt.hour

    # 3) ─── resto da lógica de Outubro (igual à sua) ───
    # (copiei tal-qual para manter o fluxo)
    novas_linhas = []
    for codigo in df['Código Postal'].unique():
        df_cp   = df[df['Código Postal'] == codigo]
        df_sept = df_cp[df_cp['Mês'] == 9]
        df_nov  = df_cp[df_cp['Mês'] == 11]
        if df_sept.empty or df_nov.empty:
            continue

        media_sept = df_sept.groupby(['Dia', 'Hora'])['Energia ativa (kWh)'].mean()
        media_nov  = df_nov.groupby(['Dia', 'Hora'])['Energia ativa (kWh)'].mean()
        df_razoes  = (
            pd.concat([media_sept.rename('Valor_setembro'),
                       media_nov.rename('Valor_novembro')], axis=1)
              .dropna()
              .reset_index()
        )
        if df_razoes.empty:
            continue

        df_razoes['Valor_outubro'] = (
            df_razoes['Valor_setembro'] *
            (1 + ((df_razoes['Valor_novembro']/df_razoes['Valor_setembro']) - 1) * 0.5)
        )

        # gera linhas de Outubro em falta
        for _, r in df_razoes.iterrows():
            if df_cp[(df_cp['Mês']==10)&(df_cp['Dia']==r['Dia'])&(df_cp['Hora']==r['Hora'])].empty:
                novas_linhas.append({
                    'Data/Hora'           : pd.Timestamp(2023,10,int(r['Dia']),int(r['Hora'])),
                    'Energia ativa (kWh)' : r['Valor_outubro'],
                    'Código Postal'       : codigo,
                    'Mês'                 : 10,
                    'Dia'                 : int(r['Dia']),
                    'Hora'                : int(r['Hora']),
                })
        # duplica 30→31 Out se necessário
        df_30 = df_cp[(df_cp['Mês']==10)&(df_cp['Dia']==30)]
        for _, r in df_30.iterrows():
            if df_cp[(df_cp['Mês']==10)&(df_cp['Dia']==31)&(df_cp['Hora']==r['Hora'])].empty:
                novas_linhas.append({
                    'Data/Hora'           : pd.Timestamp(2023,10,31,int(r['Hora'])),
                    'Energia ativa (kWh)' : r['Energia ativa (kWh)'],
                    'Código Postal'       : codigo,
                    'Mês'                 : 10,
                    'Dia'                 : 31,
                    'Hora'                : int(r['Hora']),
                })

    if novas_linhas:
        df = pd.concat([df, pd.DataFrame(novas_linhas)], ignore_index=True)

    # 4) Força o ano-alvo 2024
    df['Data/Hora'] = df['Data/Hora'].apply(lambda ts: ts.replace(year=2024))
    df['Dia_MM_DD'] = df['Data/Hora'].dt.strftime('%m-%d')

    # 5) Ordena
    return df.sort_values(['Código Postal', 'Data/Hora']).reset_index(drop=True)

def consolidar_consumo_diario(df_preparado: pd.DataFrame) -> pd.DataFrame:
    """
    Devolve um dataframe com o consumo diário médio **e** o consumo normalizado
    (baseline = média do dia 10 de Fevereiro) para cada Código Postal.
    """
    # --- 4 | Média diária de 24 horas ----------------------------------------
    df_preparado['Data'] = df_preparado['Data/Hora'].dt.date
    df_diario = (
        df_preparado
        .groupby(['Código Postal', 'Data'])['Energia ativa (kWh)']
        .mean()
        .rename('Consumo_diario_medio')
        .reset_index()
    )
    df_diario['Dia_MM_DD'] = pd.to_datetime(df_diario['Data']).dt.strftime('%m-%d')

    # --- 5 | Normalização pelo dia 10/02 -------------------------------------
    baseline = (
        df_diario[df_diario['Dia_MM_DD'] == '02-10']
        .set_index('Código Postal')['Consumo_diario_medio']
    )

    df_diario['Consumo_normalizado'] = df_diario.apply(
        lambda row: row['Consumo_diario_medio'] / baseline.get(row['Código Postal'], np.nan),
        axis=1
    )

    return df_diario[['Código Postal', 'Dia_MM_DD', 'Consumo_diario_medio', 'Consumo_normalizado']]

In [3]:
df = pd.read_csv(r"../data/e-redes/raw/consumos_horario_codigo_postal.csv", sep=";")
df.head()

,Data/Hora,Data,Hora,Código Postal,Energia ativa (kWh),Dia da Semana
0,2023-03-19T00:00:00+00:00,2023-03-19,00:00,2025,4596.739709,Domingo
1,2023-09-23T12:00:00+01:00,2023-09-23,12:00,4405,14711.438243,Sábado
2,2023-02-15T05:00:00+00:00,2023-02-15,05:00,1600,21440.512557,Quarta
3,2023-02-05T11:00:00+00:00,2023-02-05,11:00,3030,16974.037997,Domingo
4,2023-02-13T09:00:00+00:00,2023-02-13,09:00,4720,7247.291623,Segunda


In [4]:
df_preparado = preparar_dados_consumo(df)
df_consolidado = consolidar_consumo_diario(df_preparado)

In [6]:
df_consolidado.to_csv(r"../data/consumos_4_digitos.csv")

In [7]:
df_consolidado.head()

,Código Postal,Dia_MM_DD,Consumo_diario_medio,Consumo_normalizado
0,1000,01-01,11504.719819,0.666244
1,1000,01-02,12360.346279,0.715794
2,1000,01-03,16205.573580,0.938473
3,1000,01-04,16733.802727,0.969063
4,1000,01-05,16841.873659,0.975321
